In [1]:
!python3 -m pip install --upgrade pip
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Cloning into 'mergekit'...
remote: Enumerating objects: 1928, done.
remote: Counting objects: 100% (1174/1174), done.
remote: Compressing objects: 100% (505/505), done.
remote: Total 1928 (delta 920), reused 766 (delta 668), pack-reused 754
Receiving objects: 100% (1928/1928), 572.57 KiB | 9.09 MiB/s, done.
Resolving deltas: 100% (1314/1314), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0

In [2]:
from huggingface_hub import notebook_login as hfl
hfl()

In [3]:
config = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3-8B-Instruct
      layer_range: [0, 32]
    - model: NousResearch/Hermes-2-Pro-Llama-3-8B
      layer_range: [0, 32]
base_model: meta-llama/Meta-Llama-3-8B-Instruct
merge_method: slerp
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

In [4]:
with open("config.yaml", "w") as f:
    f.write(config)

In [5]:
!mergekit-yaml config.yaml output-dir --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle

config.json: 100%|█████████████████████████████| 700/700 [00:00<00:00, 4.20MB/s]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/opt/conda/lib/python3.10/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py", line 398, in cached_file
    resolved_file = hf_hub_download(
  File "/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py", line 119, in _inner_fn
    return fn(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packag